<a href="https://colab.research.google.com/github/VolkhinD/Houses/blob/main/hauses_many_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer, MinMaxScaler, OrdinalEncoder, RobustScaler
from sklearn.metrics import mean_squared_log_error as MSLE, mean_absolute_error as MAE
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector as selector
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, GridSearchCV
from xgboost import XGBRegressor
color = sns.color_palette()
sns.set_style('darkgrid')
from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import FunctionTransformer

from sklearn.linear_model import ElasticNet, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.metrics import mean_squared_error
import xgboost as xgb

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import StackingRegressor
from pandas.api.types import CategoricalDtype



In [2]:
dtypes = {'Id': 'int64', 'Condition1': 'O',
 'BldgType': 'O',
 'HouseStyle': 'O',
 'OverallCond': 'O',
 'YearBuilt': 'int64',
 'YearRemodAdd': 'int64',
 'RoofStyle': 'O',
 'RoofMatl': 'str',
 'ExterQual': 'O',
 'Heating': 'O',
 'HeatingQC': 'O',
 'CentralAir': 'O',
 'Electrical': 'O',
 'BedroomAbvGr': 'O',
 'MasVnrArea': 'float64',
 'Fireplaces': 'O',
 'GarageArea': 'float64',
 'SaleType': 'O',
 'OverallQual': 'O',
 'TotalBsmtSF': 'float64',
 'GrLivArea': 'float64',
 'KitchenQual': 'O',
 'TotRmsAbvGrd': 'O',
 'BathNum': 'float64',
 'SalePrice': 'float64'}

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = '/content/drive/MyDrive/Data/houses/chosen.csv'
path_test = '/content/drive/MyDrive/Data/houses/chosen_test.csv'

data = pd.read_csv(path, dtype=dtypes, index_col=0)
df_test = pd.read_csv(path_test, dtype=dtypes, index_col=0)
print(data.shape, df_test.shape)

(1454, 26) (1459, 25)


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Preprocessing

In [6]:
y = data.pop('SalePrice')

In [7]:
print(f"Just to check how many Null values, it's {df_test.isnull().sum().sum()}")

Just to check how many Null values, it's 0


In [8]:
cat_selector = selector(dtype_include=object)
num_selector = selector(dtype_include=np.number)
cat_selector(data)

['Condition1',
 'BldgType',
 'HouseStyle',
 'OverallCond',
 'RoofStyle',
 'RoofMatl',
 'ExterQual',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'BedroomAbvGr',
 'Fireplaces',
 'SaleType',
 'OverallQual',
 'KitchenQual']

#### Linear Preprocesser

In [9]:
def box_cox_transformer(lam):
    return FunctionTransformer(lambda x: ((x + 1)**lam - 1)/lam)

There is no missing data, so thre's no Imputer in data preprocessing

In [10]:
data.isnull().sum().sum()

0

In [90]:
scaler = StandardScaler()

transformer = box_cox_transformer(lam=0.6)
fun_transformer = FunctionTransformer(np.log1p)

num_pre = Pipeline(steps=[('transformer', transformer), ('scaler', scaler)])
num_pre_1 = Pipeline(steps=[('transformer', fun_transformer), ('scaler', scaler)])
cat_pre = Pipeline(steps=[('encoder', OneHotEncoder())])

preprocesser_1 = make_column_transformer(
    (num_pre, selector(dtype_include=np.number)),
    (cat_pre, selector(dtype_include=object)),)
preprocesser_2 = make_column_transformer(
    (num_pre_1, selector(dtype_include=np.number)),
    (cat_pre, selector(dtype_include=object)),)
preprocesser_3 = make_column_transformer(
    (scaler, selector(dtype_include=np.number)),
    (cat_pre, selector(dtype_include=object)),)



Get sparse matrix after tranformation

In [91]:
data_box_cox = preprocesser_1.fit_transform(data)
data_log = preprocesser_2.fit_transform(data)
data_scaled = preprocesser_3.fit_transform(data)

#### Tree Preprocesser

In [13]:
tree_encoder = OrdinalEncoder()
#tree_cat_pre = OrdinalEncoder(handle_unknown='use_encoded_value')
data_tree = tree_encoder.fit_transform(data)


In [14]:
data_tree = pd.DataFrame(data_tree, columns=data.columns)

## All Models

#### Tree Based Model

In [139]:
performanses = {}
index = []

In [140]:
model_xgb = XGBRegressor(gamma=0.0468, max_leaves=0, booster='gbtree',
                             learning_rate=0.05, max_depth=4,
                             n_estimators=1500,
                             random_state =16, eval_metric=MSLE, tree_method='gpu_hist') # using GPU with tree_method
score_xgb = cross_val_score(model_xgb, X=data_tree, y=y, cv=10, scoring='neg_mean_absolute_error')
print(f'XGBoost mean after 10 folds {score_xgb.mean()}, Standart deviation {score_xgb.std()}')
performanses['XGB Tree'] = [score_xgb.mean(), score_xgb.std()]

XGBoost mean after 10 folds -17050.751577851912, Standart deviation 1972.4124942620374


In [94]:
%%time
grid = {'learning_rate': [0.01, 0.05, 0.1],
       'max_depth': [3, 4, 5, 6],
       'n_estimators': [2000, 5000, 10000],
       'reg_lambda': [0.1, 0.5, 1, 4]
       }
grid_search_xgb = GridSearchCV(model_xgb, grid, scoring='neg_mean_absolute_error')
grid_search_xgb.fit(data_tree, y)

CPU times: user 1h 23min 29s, sys: 37.5 s, total: 1h 24min 7s
Wall time: 1h 23min 37s


GridSearchCV(estimator=XGBRegressor(base_score=None, booster='gbtree',
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False,
                                    eval_metric=<function mean_squared_log_error at 0x7f27ad861900>,
                                    feature_types=None, gamma=0.0468,
                                    gpu_id=None, grow_policy=None,
                                    importance_type=None,
                                    inte...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=4, max_leaves=0,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    n_estimators=1500, n_jobs=None,
                                    num_parallel_tree=None, predictor=None,
                                    random_state=16, ...),
             param_grid={'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [3, 4, 5, 6],
                         'n_estimators': [2000, 5000, 10000],
                         'reg_lambda': [0.1, 0.5, 1, 4]},
             scoring='neg_mean_absolute_error')

In [95]:
# results after hyperparameter tuning
print(grid_search_xgb.best_estimator_)
print(grid_search_xgb.best_score_)

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False,
             eval_metric=<function mean_squared_log_error at 0x7f27ad861900>,
             feature_types=None, gamma=0.0468, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.01, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
             max_leaves=0, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=2000, n_jobs=None,
             num_parallel_tree=None, predictor=None, random_state=16, ...)
-16357.490747848531


16357!!! Very good result:)



In [156]:
xgb_best = grid_search_xgb.best_estimator_
xgb_best_score = cross_val_score(xgb_best, X=data_tree, y=y, cv=5, scoring='neg_mean_absolute_error')
print(f'XGBoost score {xgb_best_score .mean()}, Standart deviation {xgb_best_score .std()}')

XGBoost score -16357.490747848531, Standart deviation 1344.6639250776066


In [154]:
rf = RandomForestRegressor(n_estimators=3000,  random_state=1)
score_rf = cross_val_score(rf, X=data_tree, y=y, cv=5, scoring='neg_mean_absolute_error')
print(f'Random Forest Regressor gives score {score_rf.mean()}, Standart deviation {score_rf.std()}')
performanses['Random Forest'] = [score_rf.mean(), score_rf.std()]

Random Forest Regressor gives score -17415.462455454614, Standart deviation 1086.81387682454


In [142]:
ada = AdaBoostRegressor(n_estimators=500, random_state=16)
score_ada = cross_val_score(ada, X=data_tree, y=y, cv=5, scoring='neg_mean_absolute_error')
print(f'AdaBoost Regressor gives score {score_ada.mean()}, Standart deviation {score_ada.std()}')
performanses['Random Forest'] = [score_ada.mean(), score_ada.std()]

Random Forest Regressor gives score -25867.840048243343, Standart deviation 1356.0578437637382


#### Linear Models

In [143]:
lin_xgb = XGBRegressor(booster='gblinear',
                             n_estimators=1500,
                             random_state=16, eval_metric=MSLE)

score_xgb_lin = cross_val_score(lin_xgb, X=data_box_cox, y=y, cv=5, scoring='neg_mean_absolute_error')

print(f'XGBoost gives score {score_xgb_lin.mean()}, Standart deviation {score_xgb_lin.std()}')
performanses["XGB Linear with Box Cox transformation"] = [score_xgb_lin.mean(), score_xgb_lin.std()]

XGBoost gives score -18858.969502419477, Standart deviation 1003.7080086169697


In [144]:
pipe_xgb_lin = Pipeline([('preprocessing', preprocesser_2), ('model', lin_xgb)])
score_xgb_lin = cross_val_score(lin_xgb, X=data_log, y=y, cv=5, scoring='neg_mean_absolute_error')

print(f'XGBoost gives score {score_xgb_lin.mean()}, Standart deviation {score_xgb_lin.std()}')
performanses["XGB Linear with Log transformation"] = [score_xgb_lin.mean(), score_xgb_lin.std()]

XGBoost gives score -20264.451067854145, Standart deviation 1138.167311940857


In [145]:
pipe_xgb_lin = Pipeline([('preprocessing', preprocesser_2), ('model', lin_xgb)])
score_xgb_lin = cross_val_score(lin_xgb, X=data_scaled, y=y, cv=5, scoring='neg_mean_absolute_error')

print(f'XGBoost gives score {score_xgb_lin.mean()}, Standart deviation {score_xgb_lin.std()}')
performanses["XGB Linear"] = [score_xgb_lin.mean(), score_xgb_lin.std()]

XGBoost gives score -18060.9269354013, Standart deviation 935.3763181900675


In [146]:
ridge = Ridge(alpha =0.0005, random_state=1)

score_ridge = cross_val_score(ridge, X=data_box_cox , y=y, cv=5, scoring='neg_mean_absolute_error')
print(f'First method gives score {score_ridge.mean()}, Standart deviation {score_ridge.std()}')
performanses["Ridge Regression with Box Cox"] = [score_ridge.mean(), score_ridge.std()]

First method gives score -18859.60489814182, Standart deviation 1005.7904825858733


In [147]:
score_ridge = cross_val_score(ridge, X=data_log, y=y, cv=5, scoring='neg_mean_absolute_error')
print(f'First method gives score {score_ridge.mean()}, Standart deviation {score_ridge.std()}')
performanses["Ridge Regression with Log Transformation"] = [score_ridge.mean(), score_ridge.std()]

First method gives score -20268.21102187543, Standart deviation 1123.9659521432698


In [148]:
score_ridge = cross_val_score(ridge, X=data_scaled, y=y, cv=5, scoring='neg_mean_absolute_error')
print(f'First method gives score {score_ridge.mean()}, Standart deviation {score_ridge.std()}')
performanses["Ridge Regression"] = [score_ridge.mean(), score_ridge.std()]

First method gives score -18047.744201239788, Standart deviation 942.9013075248196


In [149]:
ENet = ElasticNet(alpha=0.01, l1_ratio=.9, random_state=1)

score_enet = cross_val_score(ENet, X=data_box_cox, y=y, cv=10, scoring='neg_mean_absolute_error')
print(f'Elastic Net Regression gives score {score_enet.mean()}, Standart deviation {score_enet.std()}')
performanses["ElasticNet Regression with Box Cox Transformation"] = [score_enet.mean(), score_enet.std()]

Elastic Net Regression gives score -18677.56314281853, Standart deviation 1082.4529846100836


In [150]:
score_enet = cross_val_score(ENet, X=data_log, y=y, cv=10, scoring='neg_mean_absolute_error')
print(f'Elastic Net Regression gives score {score_enet.mean()}, Standart deviation {score_enet.std()}')
performanses["ElasticNet Regression with Log Transformation"] = [score_enet.mean(), score_enet.std()]

Elastic Net Regression gives score -20003.840969312572, Standart deviation 1450.5543861446708


In [151]:
score_enet = cross_val_score(ENet, X=data_scaled, y=y, cv=10, scoring='neg_mean_absolute_error')
print(f'Elastic Net Regression gives score {score_enet.mean()}, Standart deviation {score_enet.std()}')
performanses["ElasticNet Regression "] = [score_enet.mean(), score_enet.std()]

Elastic Net Regression gives score -17870.495507089432, Standart deviation 986.3671281535613


## Take a Look at All Results and choose best model for hyperparameter tuning

In [155]:
res = pd.DataFrame(performanses).T
res.columns = ["MAE maen()", "MAE std()"]
res

,MAE maen(),MAE std()
XGB Tree,-17050.751578,1972.412494
Random Forest,-17415.462455,1086.813877
XGB Linear with Box Cox transformation,-18858.969502,1003.708009
XGB Linear with Log transformation,-20264.451068,1138.167312
XGB Linear,-18060.926935,935.376318
Ridge Regression with Box Cox,-18859.604898,1005.790483
Ridge Regression with Log Transformation,-20268.211022,1123.965952
Ridge Regression,-18047.744201,942.901308
ElasticNet Regression with Box Cox Transformation,-18677.563143,1082.452985
ElasticNet Regression with Log Transformation,-20003.840969,1450.554386


Sad, but feature transformation does't help this time

## Stacking

https://scikit-learn.org/stable/auto_examples/ensemble/plot_stack_predictors.html?highlight=stacking

In [162]:
%%time

estimators = [
    ("XGBoost", model_xgb),
    ("Random Forest", rf),
]

stacking_regressor = StackingRegressor(estimators=estimators, final_estimator=XGBRegressor(random_state=1))
score_stack = cross_val_score(stacking_regressor, X=data_tree, y=y, cv=5, scoring='neg_mean_absolute_error')
print(f'Stacking Regressor gives score {score_stack.mean()}, Standart deviation {score_stack.std()}')

Stacking Regressor gives score -20067.902815240253, Standart deviation 2038.2360837224676
CPU times: user 10min 30s, sys: 2.71 s, total: 10min 33s
Wall time: 10min 31s


## Submission

In [160]:
sub_1 = pd.DataFrame()
sub_1['Id'] = df_test.index
xgb_test_pred = xgb_best_score.predict(df_test)
sub_1['SalePrice'] = xgb_test_pred
sub_1 = sub_1.set_index('Id')

AttributeError: ignored

In [159]:
sub_1.head()

,Id
0,1461
1,1462
2,1463
3,1464
4,1465


In [ ]:
sub_1.to_csv('/Users/volkhind/Desktop/github/houses/sub_1.csv')

In [ ]:
sub_2 = pd.DataFrame()
sub_2['Id'] = df_test.index
xgb_test_pred2 = xgb_best.predict(df_test)
sub_2['SalePrice'] = xgb_test_pred2
sub_2 = sub_2.set_index('Id')
sub_2.head()